<a href="https://colab.research.google.com/github/usseif97/MRNET-for-Knee-Diagnosis/blob/master/train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  print('GPU not found')
  raise SystemError('GPU device not found')


In [2]:
# importing the drive to get data files
import tensorflow as tf
import sys


from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [3]:
cd '/content/drive/My Drive/DataSet/MRNET data set/MRNet-v1.0'

/content/drive/My Drive/DataSet/MRNET data set/MRNet-v1.0


In [0]:
sys.path.append('/content/drive/My Drive/DataSet/MRNET data set/MRNet-v1.0')


In [0]:
import numpy as np
from tensorflow import keras
from tensorflow.keras import applications, layers, Input
from tensorflow.keras.models import Sequential
import keras.backend.tensorflow_backend as tfback
import pandas as pd
from matplotlib import pyplot as plt
import pickle

root_path = '/content/drive/My Drive/DataSet/MRNET data set/MRNet-v1.0'
tf.keras.backend.set_floatx('float32')


In [95]:
print("tf.version is", tf.version)
#print("tf.keras.version is:", keras.version)

def _get_available_gpus():
  """Get a list of available gpu devices (formatted as strings).
  # Returns
     A list of available GPU devices. 
  """
#global _LOCAL_DEVICES
  if tfback._LOCAL_DEVICES is None:
    devices = tf.config.list_logical_devices()
    tfback._LOCAL_DEVICES = [x.name for x in devices]
  return [x for x in tfback._LOCAL_DEVICES if 'device:gpu' in x.lower()]

tfback._get_available_gpus = _get_available_gpus
keras.backend.set_image_data_format('channels_first')


tf.version is <module 'tensorflow._api.v2.version' from '/usr/local/lib/python3.6/dist-packages/tensorflow/_api/v2/version/__init__.py'>


In [0]:
%run datagen.ipynb
%run model.ipynb

In [56]:
from timeit import default_timer as timer
start = timer()
end = timer()
print(end - start) # Time in seconds, e.g. 5.38091952400282

1.4455999917117879e-05


In [0]:
#TRAIN VGG
vgg_axial_abnormal = VGG16(root_path, 'axial', 'abnormal')
start = timer()
logs = vgg_axial_abnormal.train_model(save_model=True, X=X)
end = timer()

print(end - start)

In [0]:
#TRAIN resnet
resnet_axial_abnormal = ResNet(root_path, 'axial', 'abnormal')
start = timer()
logs = resnet_axial_abnormal.train_model(save_model=True)
end = timer()

print(end - start)

In [0]:
#TRAIN Inception
resnet_axial_abnormal = InceptionV3(root_path, 'axial', 'abnormal')
start = timer()
logs = resnet_axial_abnormal.train_model(save_model=True)
end = timer()

print(end - start)

Transfer Learning:

In [0]:
model_name = 'vgg16' #set the model
case = 'abnormal' #set the case

#Extracting Features:

#Run Once
#Extracting Features with transfer-learning to axial (Generator Mode)

extract_axial = TransferLearningModel(root_path,'axial', case, model_name, task='train')
start = timer()
extract_axial.extract_features('train', save = True)
end = timer()
print(f'EXTRACTING AXIAL FINISHED in {end - start}s') #2411s

#Extracting Features with transfer-learning to coronal

extract_axial = TransferLearningModel(root_path,'coronal', case, model_name, task='train')
start = timer()
extract_axial.extract_features('train', save = True)
end = timer()
print(f'EXTRACTING CORONAL FINISHED in {end - start}s')

#Run Once
#Extracting Features with transfer-learning to sagittal (Generator Mode)

start = timer()
extract_sagittal = TransferLearningModel(root_path,'sagittal', case, model_name, task='train')
extract_sagittal.extract_features('train', save = True)
end = timer()
print(f'EXTRACTING SAGITTAL FINISHED in {end - start}s') #5394s

#Run Once
#Extracting Features with transfer-learning to axial (Generator Mode) --TestData

extract_axial_test = TransferLearningModel(root_path,'axial', case, model_name, task='valid')
start = timer()
extract_axial_test.extract_features('valid', save = True)
end = timer()
print(f'EXTRACTING AXIAL FINISHED in {end - start}s')

#Run Once
#Extracting Features with transfer-learning to coronal

extract_coronal_test = TransferLearningModel(root_path,'coronal', case, model_name, task='valid')
start = timer()
extract_coronal_test.extract_features('valid', save = True)
end = timer()
print(f'EXTRACTING CORONAL FINISHED in {end - start}s')

#Run Once
#Extracting Features with transfer-learning to sagittal (Generator Mode)

start = timer()
extract_sagittal_test = TransferLearningModel(root_path,'sagittal', case, model_name, task='valid')
extract_sagittal_test.extract_features('valid', save = True)
end = timer()
print(f'EXTRACTING SAGITTAL FINISHED in {end - start}s') #5394s


In [0]:
#Train FC Layer with transfer-learning to axial

train_fc_axial = TransferLearningModel(root_path,'axial', case, model_name, task='train')
start = timer()
history = train_fc_axial.train_fc(save_fc = False, memory = True)
end = timer()
train_fc_axial.plot_logs()
print(f'TRAINING FC LAYER - AXIAL FINISHED in {end - start}s') #62s #692

In [0]:
#Train FC Layer with transfer-learning to coronal

train_fc_coronal = TransferLearningModel(root_path,'coronal', case, model_name, task='train')
start = timer()
history = train_fc_coronal.train_fc(save_fc = True, memory = True)
end = timer()
train_fc_axial.plot_logs()
print(f'TRAINING FC LAYER - CORONAL FINISHED in {end - start}s') #

In [0]:
#Train FC Layer with transfer-learning to sagittal

train_fc_sagittal = TransferLearningModel(root_path,'sagittal', case, model_name, task='train')
start = timer()
history = train_fc_sagittal.train_fc(save_fc = True, memory = True)
end = timer()
train_fc_axial.plot_logs()
print(f'TRAINING FC LAYER - SAGITTAL FINISHED in {end - start}s') #

In [0]:
#Evaluate One View with Transfer Learning
model_name = 'vgg16'
features_dim = 512
view = 'axial'
case = 'abnormal'

records = pd.read_csv(root_path + f"/valid-{case}.csv", header=None, names=['id', 'label'])
Y = records['label'].to_numpy()

X = np.empty((0, features_dim))

for i in range(1130, 1250):
  index = '0' * (4 - len(str(i))) + str(i)
  path = root_path + f"/valid/{view}/{model_name}/{index}.npy"
  print('loading', index, 'features')
  x = np.load(path).reshape((1, features_dim))
  X = np.concatenate((X, x), axis = 0)

train_fc_axial.fc.evaluate(X, Y)

In [0]:
#Prediction Layer

axial_predict = TransferLearningModel(root_path,'axial', case, model_name, task='train')
start = timer()
axial_predictions = axial_predict.predict_fc()
end = timer()
print(f'PREDICTING FC LAYER - AXIAL FINISHED in {end - start}s')

coronal_predict = TransferLearningModel(root_path,'coronal', case, model_name, task='train')
start = timer()
coronal_predictions = coronal_predict.predict_fc()
end = timer()
print(f'PREDICTING FC LAYER - CORONAL FINISHED in {end - start}s')

sagittal_predict = TransferLearningModel(root_path,'sagittal', case, model_name, task='train')
start = timer()
sagittal_predictions = sagittal_predict.predict_fc()
end = timer()
print(f'PREDICTING FC LAYER - SAGITTAL FINISHED in {end - start}s')

early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss',
  min_delta=0,
  patience=5,
  verbose=0, mode='auto',
  restore_best_weights=True
)
    
reduce_lr = keras.callbacks.ReduceLROnPlateau(
  monitor='val_loss', factor=0.1, patience=3, verbose=0, mode='auto',
  min_delta=0, cooldown=0, min_lr=1e-5
)
inputs = keras.Input((3,))
outputs = keras.layers.Dense(1, activation='sigmoid')(inputs)
classifier_model = keras.Model(inputs, outputs)
classifier_model.compile(
  optimizer=keras.optimizers.Adam(1e-4), loss='binary_crossentropy', metrics=[keras.metrics.AUC()]
  )

Xp = np.concatenate((axial_predictions, coronal_predictions, sagittal_predictions), axis=1) #this is the input to sigmoid function
records = pd.read_csv(root_path + f"/train-{case}.csv", header=None, names=['id', 'label'])
Y = records['label'].to_numpy()


history = classifier_model.fit(
  x=Xp, y=Y, batch_size=1, epochs=100, verbose=2, callbacks=[reduce_lr, early_stopping],
  validation_split=0.1, validation_data=None, shuffle=True,
)
train_classifier_logs = history.history

plot_logs(train_classifier_logs)

classifier_model.save(root_path + f'/models/{model_name}-auc-classifier-abnormal.h5')
with open(root_path + f'/models/{model_name-auc-classifier-abnormal-history', 'wb') as file_pi:
      pickle.dump(train_classifier_logs, file_pi)

In [0]:
#Evaluate Prediction Layer

classifier_model = keras.models.load_model(root_path + f'/models/{model_name}-auc-classifier-abnormal.h5')

axial_predict = TransferLearningModel(root_path,'axial', case, model_name, task='valid')
axial_predict.fc.summary()
start = timer()
axial_predictions = axial_predict.predict_fc()
end = timer()
print(f'PREDICTING FC LAYER - AXIAL FINISHED in {end - start}s')

coronal_predict = TransferLearningModel(root_path,'coronal', case, model_name, task='valid')
coronal_predict.fc.summary()
start = timer()
coronal_predictions = coronal_predict.predict_fc()
end = timer()
print(f'PREDICTING FC LAYER - CORONAL FINISHED in {end - start}s')

sagittal_predict = TransferLearningModel(root_path,'sagittal', case, model_name, task='valid')
sagittal_predict.fc.summary()
start = timer()
sagittal_predictions = sagittal_predict.predict_fc()
end = timer()
print(f'PREDICTING FC LAYER - SAGITTAL in {end - start}s')

Xp_val = np.concatenate((axial_predictions, coronal_predictions, sagittal_predictions), axis=1) #this is the input to sigmoid function
records = pd.read_csv(root_path + f"/valid-abnormal.csv", header=None, names=['id', 'label'])
Y_val = records['label'].to_numpy()

In [0]:
history_eval = classifier_model.evaluate(
    x=Xp_val, y=Y_val, verbose=2
)

4/4 - 0s - loss: 0.4230 - auc_24: 0.9229


ResNet Transfer Learning:

## **The next is rubbish:**

In [0]:
def load_features(root_dir = '/content/drive/My Drive/DataSet/MRNET data set/MRNet-v1.0', set = 'train', view = 'axial', model = 'VGG16'):
  X = np.empty((0, 512))
  start = 0
  end = 10 ######
  if set == 'train':
    pass
  else:
    start = 1130
    end = 1250
  for i in range(start, end):
    index = '0' * (4 - len(str(i))) + str(i)
    path = root_dir + f"/{set}/{view}/{model}/{index}.npy"
    print('loading', index)
    x = np.load(path).reshape((1, 512))
    X = np.concatenate((X, x), axis = 0)

  return X


In [0]:
def load_labels(root_dir = '/content/drive/My Drive/DataSet/MRNET data set/MRNet-v1.0', set = 'train', case = 'abnormal'):
  records = pd.read_csv(root_dir + f"/train-{case}.csv", header=None, names=['id', 'label'])
  return records['label'].to_numpy()


In [0]:
def load_input():
  pass

In [0]:
X = {}
for i in range(1):
  X[i] = load_scan(view='axial', num = i)


loading 0000


In [0]:
def save_example(root_dir, view, num, features, model, set):
  root_dir = '/content/drive/My Drive/DataSet/MRNET data set/MRNet-v1.0'
  index = '0' * (4 - len(str(num))) + str(num)
  np.save(open(f"{root_dir}/{set}/{view}/{model}/{index}.npy", 'wb'), features)

In [0]:

features = {}
for i in range(1):
  print(i)
  x = X[i]
  x = applications.vgg16.preprocess_input(x)
  features[i] = vgg_fx.predict(
      x, batch_size=tf.constant(x.shape[0]), verbose=0,
  )
  

0


In [0]:
for example_num, example_features in features.items():
  print('saving', example_num)
  save_example('', 'axial', example_num, example_features, 'VGG16', 'train')

In [0]:
#classifier
#inputs = keras.Input(shape=(512))
#outputs = layers.Dense(1, activation='relu')(inputs, training=False)
#vgg_fc = keras.Model(inputs, outputs)
#vgg_fc.summary()
#vgg_fx.compile()
vgg_fc = Sequential()
vgg_fc.add(Input(shape = (512,)))
vgg_fc.add(layers.Dense(1, activation='relu'))
vgg_fc.summary()
vgg_fc.compile(
    optimizer=keras.optimizers.Adam(1e-5), loss='binary_crossentropy', metrics=['accuracy']
)

early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss',
                              min_delta=0,
                              patience=20,
                              verbose=0, mode='auto',
                              restore_best_weights=True)

Y_train = load_labels(case='abnormal')

X_train = load_features()
vgg_fc.fit(
    x=X_train, y=Y_train[:50], batch_size=1, epochs=10, verbose=2, callbacks=[early_stopping],
    validation_split=0.0, validation_data=None, shuffle=True,
)


#callbacks=[early_stopping],

In [0]:
#Train with custom data generator

vgg_fc = Sequential()
vgg_fc.add(Input(shape = (512,)))
vgg_fc.add(layers.Dense(1, activation='relu'))
vgg_fc.summary()
vgg_fc.compile(
    optimizer=keras.optimizers.Adam(1e-5), loss='binary_crossentropy', metrics=['accuracy']
)

early_stopping = keras.callbacks.EarlyStopping(monitor='loss',
                              min_delta=0,
                              patience=20,
                              verbose=0, mode='auto',
                              restore_best_weights=True)

params = {'shape': (512),
          'batch_size': 1,
          'shuffle': False,
          'extractor': 'VGG16'}

training_generator = MRNetDataGenerator(**params)

vgg_fc.fit(
    training_generator, epochs=50, verbose=2, callbacks=[early_stopping],
    use_multiprocessing = False
)

In [0]:
root_dir = '/content/drive/My Drive/DataSet/MRNET data set/MRNet-v1.0'
vgg_fc.save(root_dir + '/vgg16-fc-abnormal.h5')

In [0]:
#classifier
vgg_full = Sequential()
vgg_full.add(vgg_fx)
vgg_full.add(vgg_fc)
vgg_full.summary()
vgg_full.compile(
    optimizer=keras.optimizers.Adam(1e-5), loss='binary_crossentropy', metrics=['accuracy']
)

In [0]:
inputs = keras.Input(shape=(256,256,3))
x = vgg_fx(inputs
outputs = vgg_fc(x)
vgg_full = keras.Model(inputs, outputs)
vgg_full.summary()
vgg_full.compile()


In [0]:
X = {}
for i in range(1130, 1250):
  X[i] = load_scan(set = 'valid', view='axial', num = i)


In [0]:
features = {}
for i in range(1130, 1250):
  print(i)
  x = X[i]
  applications.vgg16.preprocess_input(x)
  features[i] = vgg_fx.predict(
      x, batch_size=tf.constant(x.shape[0]), verbose=0,
  )
for example_num, example_features in features.items():
  print('saving', example_num)
  save_example('', 'axial', example_num, example_features, 'VGG16', 'valid')

In [0]:
Y_valid = load_labels(case='abnormal', set = 'valid')

X_valid = load_features(set='valid')
vgg_fc.evaluate(
    x=X_train, y=Y_train, batch_size=1, verbose=2)

In [0]:
inputs = keras.Input(shape=(None,256, 256, 3))
x = layers.Flatten()(inputs)
mo = keras.Model(inputs, x)
mo.summary()
mo.compile()

arrr = np.random.rand(1, 10,256,256,3)

moo = mo(arrr)
moo.shape

In [0]:
tei = keras.Input(shape=(3, 256,256))
te = applications.vgg16.preprocess_input(tei)
moo = keras.Model(tei,te)
moo.summary()
moo.compile()
inn = np.ones((2,3,256,256))

for j, i in enumerate(inn):
  print(i.shape, j)

Model: "model_73"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_109 (InputLayer)       [(None, 3, 256, 256)]     0         
_________________________________________________________________
tf_op_layer_strided_slice_13 [(None, 3, 256, 256)]     0         
_________________________________________________________________
tf_op_layer_BiasAdd_13 (Tens [(None, 3, 256, 256)]     0         
Total params: 0
Trainable params: 0
Non-trainable params: 0
_________________________________________________________________
(3, 256, 256) 0
(3, 256, 256) 1


In [0]:
def load_scan(task, view, index, root_path = root_path):
    index_extended = '0' * (4 - len(str(index))) + str(index)
    path = root_path + f"/{task}/{view}/{index_extended}.npy"
    print(f'loading example: {index}')
    x = np.load(path)
    x = np.stack((x,)*3, axis=1)
    return x

def load_view(task, view, start, end, root_path = root_path):
  data_size = end - start
  X = {}
  for i, index in enumerate(range(start, end)):
    X[i] = load_scan(task, view, index, root_path)
  return X

In [0]:
 def save_example2(num, features, model):
    index = '0' * (4 - len(str(num))) + str(num)
    np.save(open(f"{root_path}/{model.task}/{model.view}/VGG16/{index}.npy", 'wb'), features)

for example_num, example_features in model.features.items():
  print('saving', example_num)
  save_example2(example_num, example_features, model)
  